In [1]:
%autosave 0

Autosave disabled


In [2]:
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent))
from os import remove


from datetime import date, timedelta
import codecs
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

from os.path import join, basename

from bulletin import default_input, default_output, root
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_labels, normalize_text, date_hash, normalize_number
from bulletin.utils import static
from bulletin.utils.xls_ import fit_cols

from datetime import datetime
import pyminizip as pz

today = datetime.today()
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)

municipios = Municipios()
municipios['mun_resid'] = municipios['municipio'].apply(normalize_text)
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'].apply(normalize_text) + '/' + municipios['uf']


---------------------------------------------------------------

root: C:\sesa\dev\bulletin
parent: C:\sesa\dev
default_input: C:\sesa\dev\input
default_output: C:\sesa\dev\output

---------------------------------------------------------------



In [3]:
mb = Metabase()
intervalo = "classificacao_final = 2 AND excluir_ficha = 2 AND status_notificacao in (1,2) AND ((data_notificacao >= NOW() - INTERVAL '7 DAY') or (data_liberacao >= NOW() - INTERVAL '7 DAY') or (updated_at >= NOW() - INTERVAL '7 DAY') or (data_coleta >= NOW() - INTERVAL '7 DAY') or (data_encerramento >= NOW() - INTERVAL '7 DAY'))"
mb.generate_notifica_query('update_notifica', where=intervalo)
mb.list_sql_files()

limit: 100000
Cookie:'13646********-****-****-****-************27d89'

sql_files:
	0: all_notifica
	1: coisa
	2: comorbidades
	3: diario
	4: instituicao
	5: new_query
	6: old_diario
	7: query_linda
	8: update_notifica
	9: usecols_notifica

sql_results:
	0: comorbidades_len
	1: diario_len
	2: instituicao_len
	3: query_linda_len
	4: update_notifica_len
Select 44 columns

sql_files:
	0: all_notifica
	1: coisa
	2: comorbidades
	3: diario
	4: instituicao
	5: new_query
	6: old_diario
	7: query_linda
	8: update_notifica
	9: usecols_notifica


In [4]:
update_notifica = mb.download_notificacao('update_notifica', load=True, normalize=True)
update_notifica.save()
len(update_notifica.df)

Download update_notifica

query_size: 41885

Reading C:\sesa\dev\input\queries\tmp\update_notifica_0_100000.csv
normalize notifica
Attrb
41887
reading Notifica time elepsed:00:01:16
Download query time elepsed:00:01:16


41887

In [5]:
notifica = Notifica()
notifica.load()
notifica.update(update_notifica.df)
notifica.save()
len(notifica.df)

novas_notificacoes 0
possiveis_atualizacoes 41887


1192685

In [6]:
casos_notifica = notifica.df.drop_duplicates('hash', keep='last')
casos_notifica.loc[casos_notifica['data_liberacao'].notna(),'hash_lib'] = casos_notifica.loc[casos_notifica['data_liberacao'].notna(),'paciente'].apply(normalize_hash) + casos_notifica.loc[casos_notifica['data_liberacao'].notna(),'data_liberacao'].apply(date_hash)
casos_notifica.loc[casos_notifica['data_diagnostico'].notna(),'hash_diag'] = casos_notifica.loc[casos_notifica['data_diagnostico'].notna(),'paciente'].apply(normalize_hash) + casos_notifica.loc[casos_notifica['data_diagnostico'].notna(),'data_diagnostico'].apply(date_hash)
casos_notifica = casos_notifica.loc[casos_notifica['data_diagnostico'].notna()]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [7]:
casos_notifica = pd.merge(casos_notifica.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','rs','mun_resid','uf','municipio']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})
casos_notifica = pd.merge(casos_notifica.rename(columns={'ibge_unidade_notifica':'ibge'}),municipios[['ibge','mun_resid']].rename(columns={'mun_resid':'mun_atend'}),on='ibge',how='left').rename(columns={'ibge':'ibge_unidade_notifica'})
casos_notifica = casos_notifica.loc[casos_notifica['rs'].notna()]
casos_notifica = casos_notifica.loc[casos_notifica['pais_residencia']==1]

casos_notifica = pd.merge(casos_notifica,pd.read_csv(join(root,'resources','tables','exame.csv')).rename(columns={'exame':'exame_name','id':'exame'}), on='exame', how='left')

casos_notifica = casos_notifica.loc[casos_notifica['sexo']!=3]
casos_notifica.loc[casos_notifica['sexo']==1,'sexo'] = 'M'
casos_notifica.loc[casos_notifica['sexo']==2,'sexo'] = 'F'

casos_notifica

,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_less,hash_more,hash_mae,hash_nasc,hash_lib,hash_diag,rs,mun_resid,uf,municipio,mun_atend,exame_name
0,79085,2020-04-27,2020-04-06,2.0,1.0,2021-04-07,None,2.0,1.0,1.0,2020-04-06,3.0,None,00782773966,NaN,NaN,NANDREIA APARECIDA DA SILVA DE,F,1981-03-24,MARIA DE LOURDES DA ROCHA DA SILVA,99.0,5.0,PASTORA,NaN,NaN,1.0,,41.0,410480.0,NaN,RUA MEDICINA,UNIVERSITARIO,41.0,410480.0,NaT,2020-03-27,2020-03-27,2020-03-28,-1.000000e+00,12.0,21.0,1.0,,2020-03-23,2020-03-27,39.0,NANDREIAAPARECIDADASILVADE39410480,NANDREIAAPARECIDADASILVADE39410480,NANDREIAAPARECIDADASILVADE38410480,NANDREIAAPARECIDADASILVADE40410480,NANDREIAAPARECIDADASILVADEMARIADELOURDESDAROCH...,NANDREIAAPARECIDADASILVADE24031981,NANDREIAAPARECIDADASILVADE28032020,NANDREIAAPARECIDADASILVADE27032020,10.0,CASCAVEL,PR,Cascavel,CASCAVEL,One Step COVID-2019 Test
1,79096,2020-04-27,2020-05-20,2.0,1.0,2020-06-07,None,2.0,1.0,1.0,2020-05-20,3.0,704501695833220,06623607960,NaN,0.0,GABRIELA APARECIDA PONZIO,F,1988-12-02,MARIA ANTONIA MAURO PONZIO,1.0,1.0,aux. administrativo /coordenação secretaria,0.0,NaN,1.0,,41.0,412370.0,87910000,rua princesa isabel,centro,41.0,412370.0,2020-05-04,2020-05-04,2020-05-06,2020-05-07,2.014010e+11,9.0,1.0,1.0,LABORATORIO CENTRAL DO ESTADO,2020-04-22,2020-05-04,31.0,GABRIELAAPARECIDAPONZIO31412370,GABRIELAAPARECIDAPONZIO31412370,GABRIELAAPARECIDAPONZIO30412370,GABRIELAAPARECIDAPONZIO32412370,GABRIELAAPARECIDAPONZIOMARIAANTONIAMAUROPONZIO,GABRIELAAPARECIDAPONZIO02121988,GABRIELAAPARECIDAPONZIO07052020,GABRIELAAPARECIDAPONZIO04052020,14.0,SANTA ISABEL DO IVAI,PR,Santa Isabel do Ivaí,SANTA ISABEL DO IVAI,Vírus Respiratório
2,79103,2020-04-27,2020-05-11,2.0,1.0,2020-04-27,None,2.0,1.0,1.0,NaT,3.0,None,04283213993,NaN,0.0,ALESSANDRA FERNANDES BRITES,F,1983-04-23,MARIA FERNANDES NUNES,99.0,5.0,PEDAGOGA,NaN,NaN,1.0,,41.0,412125.0,85888000,JK DE OLIVEIRA,CENTRO,41.0,412125.0,2020-04-22,NaT,NaT,NaT,-1.000000e+00,12.0,21.0,2.0,,2020-04-08,2020-04-27,37.0,ALESSANDRAFERNANDESBRITES37412125,ALESSANDRAFERNANDESBRITES37412125,ALESSANDRAFERNANDESBRITES36412125,ALESSANDRAFERNANDESBRITES38412125,ALESSANDRAFERNANDESBRITESMARIAFERNANDESNUNES,ALESSANDRAFERNANDESBRITES23041983,NaN,ALESSANDRAFERNANDESBRITES27042020,9.0,RAMILANDIA,PR,Ramilândia,RAMILANDIA,One Step COVID-2019 Test
3,79106,2020-04-27,2020-11-04,2.0,1.0,2020-11-04,None,2.0,1.0,1.0,2020-10-29,3.0,None,25503235720,NaN,0.0,LUIZ CARLOS DE LIMA,M,1951-04-19,NEIDA DE OLIVEIRA LIMA,1.0,0.0,Não Informado,0.0,NaN,1.0,,41.0,411370.0,NaN,RUA JOSE BELETTI,JARDIM CHAMONIX,41.0,411370.0,2020-10-27,2020-10-24,2020-10-27,2020-10-27,2.017530e+11,9.0,27.0,1.0,UNIMED,2020-10-20,2020-10-24,69.0,LUIZCARLOSDELIMA69411370,LUIZCARLOSDELIMA69411370,LUIZCARLOSDELIMA68411370,LUIZCARLOSDELIMA70411370,LUIZCARLOSDELIMANEIDADEOLIVEIRALIMA,LUIZCARLOSDELIMA19041951,LUIZCARLOSDELIMA27102020,LUIZCARLOSDELIMA24102020,17.0,LONDRINA,PR,Londrina,LONDRINA,"COVID-19, Biologia Molecular"
4,79151,2020-04-27,2020-04-19,2.0,1.0,2021-04-07,None,2.0,1.0,1.0,2020-04-19,3.0,None,02304278981,NaN,NaN,FABIANO ROSA AGOSTINHO,M,1977-05-05,MARIA ODETE ROSA RODRIGUES,1.0,1.0,medico,2251.0,NaN,1.0,,41.0,410480.0,NaN,rua eca de queiroz,alto alegre,41.0,410480.0,NaT,2020-04-06,2020-04-06,2020-04-08,-1.000000e+00,12.0,21.0,1.0,UNILABOR,2020-04-05,2020-04-06,42.0,FABIANOROSAAGOSTINHO42410480,FABIANOROSAAGOSTINHO42410480,FABIANOROSAAGOSTINHO41410480,FABIANOROSA

In [61]:
cc = CasosConfirmados()
cc.load(f"cc_{ontem.strftime('%d_%m_%Y')}")

databases: ['cc_11_06_2021', 'cc_15_06_2021', 'cc_16_06_2021', 'cc_17_06_2021', 'cc_18_06_2021']
Load C:\sesa\dev\bulletin\database\casos_confirmados\cc_17_06_2021.pkl
loading Casos Confirmados from pkl time elepsed:00:00:03


In [62]:
casos_confirmados = cc.df
casos_confirmados

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
0,8838,-1,PR,410550,410550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaT,1,2020-04-03,NaT,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020
1,10932,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07,1,NaT,NaT,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU12032020
2,10933,-1,PR,410690,410690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08,1,NaT,NaT,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR12032020
3,11470,-1,PR,410690,410690,GABRIELA FUGANTI CASAGRANDE,F,26,Genoprimer,2020-03-16,2020-03-12,2020-03-11,1,NaT,NaT,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE16032020
4,11492,-1,PR,410690,410690,GABRIELA SCHNEIDER GUGELMIN,F,26,Genoprimer,2020-03-18,2020-03-12,2020-03-15,1,NaT,NaT,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN18032020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170627,1257727,-1,PR,411580,411580,ZULMA ARLETE DA SILVEIRA,F,63,27,2021-05-17,2021-06-17,2021-05-11,1,2021-05-20,2021-06-17,ZULMAARLETEDASILVEIRA63411580,ZULMAARLETEDASILVEIRA62411580,ZULMAARLETEDASILVEIRA64411580,ZULMAARLETEDASILVEIRA63411580,ZULMAARLETEDASILVEIRA62411580,ZULMAARLETEDASILVEIRA64411580,ZULMAARLETEDASILVEIRA17052021
1170628,1261774,-1,PR,412570,412570,ZULMA BEATRIZ LOPEZ DE FERREIRA,F,49,27,2021-06-15,2021-06-17,2021-06-10,3,NaT,NaT,ZULMABEATRIZLOPEZDEFERREIRA49412570,ZULMABEATRIZLOPEZDEFERREIRA48412570,ZULMABEATRIZLOPEZDEFERREIRA50412570,ZULMABEATRIZLOPEZDEFERREIRA49412570,ZULMABEATRIZLOPEZDEFERREIRA48412570,ZULMABEATRIZLOPEZDEFERREIRA50412570,ZULMABEATRIZLOPEZDEFERREIRA15062021
1170629,1259702,-1,PR,410430,410430,ZULMIR LUIZ PASINI,M,76,1,2021-06-02,2021-06-17,2021-05-30,1,2021-06-14,2021-06-17,ZULMIRLUIZPASINI76410430,ZULMIRLUIZPASINI75410430,ZULMIRLUIZPASINI77410430,ZULMIRLUIZPASINI76410430,ZULMIRLUIZPASINI75410430,ZULMIRLUIZPASINI77410430,ZULMIRLUIZPASINI02062021
1170630,1258691,-1,PR,411300,410550,ZULMIRA APARECIDA MOREZZI GUIETI,F,69,167,2021-06-15,2021-06-17,2021-06-13,3,NaT,NaT,ZULMIRAAPARECIDAMOREZZIGUIETI69411300,ZULMIRAAPARECIDAMOREZZIGUIETI68411300,ZULMIRAAPARECIDAMOREZZIGUIETI70411300,ZULMIRAAPARECIDAMOREZZIGUIETI69410550,ZULMIRAAPARECIDAMOREZZIGUIETI68410550,ZULMIRAAPARECIDAMOREZZIGUIETI70410550,ZULMIRAAPARECIDAMOREZZIGUIETI15062021


In [63]:
casos_confirmados[['identificacao','evolucao']].groupby('evolucao').count()

,identificacao
evolucao,
1,824521
2,29177
3,316934


In [64]:
casos_confirmados = pd.merge(casos_confirmados.rename(columns={'ibge_resid':'ibge'}),municipios,how='left',on='ibge').rename(columns={'ibge':'ibge_resid'})

In [65]:
# duplicados = casos_confirmados.loc[casos_confirmados.duplicated('hash',keep='first')]
# casos_confirmados = casos_confirmados.drop(index=duplicados.index)

# duplicados_hash_diag = casos_confirmados.loc[casos_confirmados.duplicated('hash_diag',keep='first')]
# duplicados_hash_diag = duplicados_hash_diag.loc[duplicados_hash_diag['id_notifica']!=-1]
# casos_confirmados = casos_confirmados.drop(index=duplicados_hash_diag.index)

# duplicados = duplicados.append(duplicados_hash_diag)

# duplicados_hash_diag = casos_confirmados.loc[casos_confirmados.duplicated('hash_diag',keep='first')]
# duplicados_hash_diag = duplicados_hash_diag.iloc[:250-len(duplicados)]
# casos_confirmados = casos_confirmados.drop(index=duplicados_hash_diag.index)

# duplicados = duplicados.append(duplicados_hash_diag)

# duplicados = pd.merge(duplicados.rename(columns={'ibge_resid':'ibge'}),municipios,on='ibge',how='left')

# duplicados

In [66]:
# duplicados.to_pickle(join(default_output,'exclusoes.pkl'))
# duplicados.to_excel(join(default_output,'exclusoes.xlsx'))
# duplicados[['identificacao','evolucao','rs']].groupby(['rs','evolucao']).count()

In [67]:
novos_casos = casos_notifica.loc[~(
    (casos_notifica['id'].isin(casos_confirmados['id_notifica'])) |
    (casos_notifica['hash'].isin(casos_confirmados['hash'])) |
    (casos_notifica['hash'].isin(casos_confirmados['hash_less'])) |
    (casos_notifica['hash'].isin(casos_confirmados['hash_more'])) |
    (casos_notifica['hash_atend'].isin(casos_confirmados['hash_atend'])) |
    (casos_notifica['hash_atend'].isin(casos_confirmados['hash_less_atend'])) |
    (casos_notifica['hash_atend'].isin(casos_confirmados['hash_more_atend'])) |
    (casos_notifica['hash_diag'].isin(casos_confirmados['hash_diag'])) |
    (casos_notifica['hash_lib'].isin(casos_confirmados['hash_diag']))
)].copy()

novos_casos['data_com'] = pd.to_datetime(date.today())
novos_casos = novos_casos.sort_values(['data_diagnostico','paciente'])

novos_casos

,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_less,hash_more,hash_mae,hash_nasc,hash_lib,hash_diag,rs,mun_resid,uf,municipio,mun_atend,exame_name,data_com
2183,127974,2020-05-14,2020-05-14,2,4,2020-06-20,None,2,4,3,NaT,3,None,87909812949,NaN,0,JOSE EDUARDO ARRUDA HAGEMEYER,M,1972-02-04,MARIA PAULINA DE MOURA A HAGEMEY,99,5,,nan,nan,1,,35,353.470,19906205,RUA ABRAAO ABUJANRA,JD PRIMAVERA,35,353.470,NaT,2020-05-05,NaT,NaT,-1,0,-1,1,,2020-05-10,2020-05-05,48,JOSEEDUARDOARRUDAHAGEMEYER48353470,JOSEEDUARDOARRUDAHAGEMEYER48353470,JOSEEDUARDOARRUDAHAGEMEYER47353470,JOSEEDUARDOARRUDAHAGEMEYER49353470,JOSEEDUARDOARRUDAHAGEMEYERMARIAPAULINADEMOURAA...,JOSEEDUARDOARRUDAHAGEMEYER04021972,NaN,JOSEEDUARDOARRUDAHAGEMEYER05052020,99,OURINHOS/SP,SP,Ourinhos,OURINHOS/SP,NaN,2021-06-18
1174102,165377,2020-05-29,2021-06-17,2,2,2021-06-17,30743290,2,1,2,2021-05-21,3,704600670789129,74873164915,NaN,nan,LURDES FATIMA HALABURA,F,1966-08-14,JULIA WUDARSKI HALABURA,1,nan,Não Informado,nan,nan,1,,41,410.690,81925620,GABRIEL RIBEIRO,SITIO CERCADO,41,410.690,NaT,NaT,NaT,NaT,210.206.156.629,9,27,1,,2020-05-27,2020-05-29,53,LURDESFATIMAHALABURA53410690,LURDESFATIMAHALABURA53410690,NaN,NaN,LURDESFATIMAHALABURAJULIAWUDARSKIHALABURA,LURDESFATIMAHALABURA14081966,NaN,LURDESFATIMAHALABURA29052020,2,CURITIBA,PR,Curitiba,CURITIBA,"COVID-19, Biologia Molecular",2021-06-18
1178864,263956,2020-06-09,2021-06-18,2,1,2021-06-18,None,2,1,1,2020-07-06,3,None,02639618938,NaN,nan,ELIANE SIQUEIRA DA SILVA,F,1976-05-05,MARIA AA,99,1,,nan,nan,1,,41,411.420,86975000,RUA CAETANO MUNHOZ DA ROCHA,CENTRO,41,411.420,NaT,NaT,NaT,NaT,-1,12,21,2,,2020-05-28,2020-06-09,44,ELIANESIQUEIRADASILVA44411420,ELIANESIQUEIRADASILVA44411420,NaN,NaN,ELIANESIQUEIRADASILVAMARIAAA,ELIANESIQUEIRADASILVA05051976,NaN,ELIANESIQUEIRADASILVA09062020,15,MANDAGUARI,PR,Mandaguari,MANDAGUARI,One Step COVID-2019 Test,2021-06-18
12523,203592,2020-06-12,2020-07-29,2,1,2020-09-16,28594353,2,1,2,2020-06-16,3,None,None,NaN,0,JOSE ALBERICO DE LIMA,M,1970-10-07,GUIOMAR DELFINA DE LIMA,99,0,Não Informado,0,nan,1,,26,260.420,55400000,RUA MAURO VANDERLEI,NOVA CATENDE,41,410.180,2020-06-13,2020-06-12,2020-06-13,2020-06-16,200.207.009.126,9,27,1,INSTITUTO DE BIOLOGIA MOLECULAR DO PARANA - IBMP,2020-06-01,2020-06-12,49,JOSEALBERICODELIMA49260420,JOSEALBERICODELIMA49410180,JOSEALBERICODELIMA48260420,JOSEALBERICODELIMA50260420,JOSEALBERICODELIMAGUIOMARDELFINADELIMA,JOSEALBERICODELIMA07101970,JOSEALBERICODELIMA16062020,JOSEALBERICODELIMA12062020,99,CATENDE/PE,PE,Catende,ARAUCARIA,"COVID-19, Biologia Molecular",2021-06-18
1174105,207897,2020-06-14,2021-06-17,2,1,2021-06-17,None,2,5,1,2021-06-14,3,None,01636241000,NaN,nan,DIEGO TEIXEIRA DE ALMEIDA,M,1993-11-09,GENECI TEIXEIRA,99,nan,Não Informado,nan,nan,1,,43,431.750,NaN,NaN,NaN,41,410.765,NaT,NaT,NaT,NaT,-1,nan,-1,3,,2020-06-13,2020-06-14,26,DIEGOTEIXEIRADEALMEIDA26431750,DIEGOTEIXEIRADEALMEIDA26410765,NaN,NaN,DIEGOTEIXEIRADEALMEIDAGENECITEIXEIRA,DIEGOTEIXEIRADEALMEIDA09111993,NaN,DIEGOTEIXEIRADEALMEIDA14062020,99,SANTO ANGELO/RS,RS,Santo Ângelo,FAZENDA RIO GRANDE,NaN,2021-06-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1178453,4300487,2021-06-18,NaT,1,1,2021-06-18,None,2,1,3,NaT,3,700206969782525,None,NaN,n

In [68]:
obitos_notifica = casos_notifica.loc[casos_notifica['evolucao']==2]
obitos_casos = casos_confirmados.loc[casos_confirmados['evolucao']==2]

novos_obitos = obitos_notifica.loc[~(
    (obitos_notifica['id'].isin(obitos_casos['id_notifica'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash_less'])) |
    (obitos_notifica['hash'].isin(obitos_casos['hash_more'])) |
    (obitos_notifica['hash_atend'].isin(obitos_casos['hash_atend'])) |
    (obitos_notifica['hash_atend'].isin(obitos_casos['hash_less_atend'])) |
    (obitos_notifica['hash_atend'].isin(obitos_casos['hash_more_atend'])) |
    (obitos_notifica['hash_diag'].isin(obitos_casos['hash_diag']))
)]

novos_obitos = novos_obitos.loc[novos_obitos['data_cura_obito'].notna()]
novos_obitos = novos_obitos.loc[novos_obitos['hash'].isin(casos_confirmados['hash'].tolist() + novos_casos['hash'].tolist())]
novos_obitos = novos_obitos.sort_values(['paciente'])

novos_obitos = novos_obitos.loc[novos_obitos['data_cura_obito'].notna()]
novos_obitos

,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_less,hash_more,hash_mae,hash_nasc,hash_lib,hash_diag,rs,mun_resid,uf,municipio,mun_atend,exame_name
973787,3655337,2021-05-12,2021-05-26,2,1,2021-05-26,313357239,2,1,2,2021-05-24,3,705206410757275,33280517915,NaN,nan,ADAIR CAPOANI,M,1959-05-21,CLARICE CAPOANI,99,nan,,nan,nan,1,,41,410.840,NaN,ANTONIO DE PAIVA CANTELMO,INDUSTRIAL,41,410.840,2021-05-20,2021-05-19,2021-05-21,2021-05-21,210.801.026.754,9,1,1,LABORATORIO CENTRAL DO ESTADO,2021-05-10,2021-05-19,61,ADAIRCAPOANI61410840,ADAIRCAPOANI61410840,ADAIRCAPOANI60410840,ADAIRCAPOANI62410840,ADAIRCAPOANICLARICECAPOANI,ADAIRCAPOANI21051959,ADAIRCAPOANI21052021,ADAIRCAPOANI19052021,8,FRANCISCO BELTRAO,PR,Francisco Beltrão,FRANCISCO BELTRAO,Vírus Respiratório
1138969,4163368,2021-06-10,2021-06-10,2,1,2021-06-10,None,2,1,2,2021-06-10,3,706207031020168,01090746962,NaN,nan,ADELAR ROQUE GARCIA,M,1964-04-11,FLORENTINA UKACHESKI GARCIA,99,nan,,nan,nan,1,,41,411.580,85884000,PARA,SAO CRISTOVAO,41,411.605,2021-06-09,2021-06-09,NaT,NaT,-1,12,167,2,,2021-05-11,2021-06-09,57,ADELARROQUEGARCIA57411580,ADELARROQUEGARCIA57411605,ADELARROQUEGARCIA56411580,ADELARROQUEGARCIA58411580,ADELARROQUEGARCIAFLORENTINAUKACHESKIGARCIA,ADELARROQUEGARCIA11041964,NaN,ADELARROQUEGARCIA09062021,9,MEDIANEIRA,PR,Medianeira,MISSAL,Teste Rápido Antígeno (Imunocromatografia)
1171723,4276834,2021-06-16,2021-06-16,2,1,2021-06-16,31831896,2,1,2,2021-06-15,3,707103894774720,65372085991,NaN,nan,ADELIA ANDRADE DE CARVALHO,F,1954-07-14,HELLY DE ANDRADE CARVALHO,1,nan,,nan,nan,1,,41,411.370,86.039.400,Rua Maria Teixeira Coutinho,Gaion,41,411.370,NaT,NaT,NaT,NaT,-1,9,27,1,,2021-05-08,2021-06-16,66,ADELIAANDRADEDECARVALHO66411370,ADELIAANDRADEDECARVALHO66411370,NaN,NaN,ADELIAANDRADEDECARVALHOHELLYDEANDRADECARVALHO,ADELIAANDRADEDECARVALHO14071954,NaN,ADELIAANDRADEDECARVALHO16062021,17,LONDRINA,PR,Londrina,LONDRINA,"COVID-19, Biologia Molecular"
1093389,4015050,2021-06-02,2021-06-14,2,1,2021-06-14,30781724,2,1,2,2021-06-14,3,704006852736161,33429790972,NaN,nan,ADELINO FERNANDES ROCHA,M,1943-07-12,REGINA FERNANDES DA ROCHA,1,nan,,nan,nan,1,,41,410.830,85854410,Avenida Pérola,Parque Ouro Verde,41,410.830,2021-06-01,2021-06-01,NaT,NaT,-1,9,27,1,,2021-05-21,2021-06-01,77,ADELINOFERNANDESROCHA77410830,ADELINOFERNANDESROCHA77410830,ADELINOFERNANDESROCHA76410830,ADELINOFERNANDESROCHA78410830,ADELINOFERNANDESROCHAREGINAFERNANDESDAROCHA,ADELINOFERNANDESROCHA12071943,NaN,ADELINOFERNANDESROCHA01062021,9,FOZ DO IGUACU,PR,Foz do Iguaçu,FOZ DO IGUACU,"COVID-19, Biologia Molecular"
931851,3526204,2021-05-05,2021-06-02,2,1,2021-06-02,None,2,2,2,2021-05-13,3,702301174934911,03225867985,NaN,nan,ADELMO MOREIRA DA SILVA,M,1977-01-14,MARIA LIBERACI DA SILVA,3,nan,ATENDENTE MERCADO SAO MIGUEL,nan,nan,1,,41,410.850,84660000,ARLINDO LAMELL,CENTRO,41,410.850,NaT,NaT,NaT,NaT,-1,nan,-1,3,,2021-05-01,2021-05-05,44,ADELMOMOREIRADASILVA44410850,ADELMOMOREIRADASILVA44410850,ADELMOMOREIRADASILVA43410850,ADELMOMOREIRADASILVA45410850,ADELMOMOREIRADASILVAMARIALIBERACIDASILVA,ADELMOMOREIRADASILVA14011977,NaN,ADELMOMOREIRADASILVA05052021,6,GENERAL CARNEIRO,PR,General Carneiro,GENERAL CARNEIRO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906865,3447692,2

In [69]:
novos_10dias = novos_obitos.loc[novos_obitos['data_cura_obito']>=today-timedelta(10)]
retroativos = novos_obitos.loc[novos_obitos['data_cura_obito']<today-timedelta(10)]

novos_10dias_sample = novos_10dias
retroativos_sample = retroativos.sample(n=313,random_state=1)

novos_obitos = pd.concat([retroativos_sample,novos_10dias_sample])
novos_obitos

,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_less,hash_more,hash_mae,hash_nasc,hash_lib,hash_diag,rs,mun_resid,uf,municipio,mun_atend,exame_name
962896,3622004,2021-05-11,2021-06-02,2,10,2021-06-02,307583708,2,1,2,2021-05-24,3,702306173383614,92902049900,NaN,nan,EUGENIA AMABILE MANFRIM,F,1937-05-28,BARBERA TONELLO,99,5,APOSENTADO,nan,nan,1,,41,412.250,87320000,RONCADOR,KM 35 // SITIO MANFRIM,41,411.520,2021-05-04,2021-05-04,2021-05-04,2021-05-04,1.605.854,9,27,1,LABORATORIO NUCLEO DIAGNOSTICO,2021-05-01,2021-05-04,83,EUGENIAAMABILEMANFRIM83412250,EUGENIAAMABILEMANFRIM83411520,EUGENIAAMABILEMANFRIM82412250,EUGENIAAMABILEMANFRIM84412250,EUGENIAAMABILEMANFRIMBARBERATONELLO,EUGENIAAMABILEMANFRIM28051937,EUGENIAAMABILEMANFRIM04052021,EUGENIAAMABILEMANFRIM04052021,11,RONCADOR,PR,Roncador,MARINGA,"COVID-19, Biologia Molecular"
1050155,3881279,2021-05-26,NaT,1,1,2021-06-02,None,2,1,2,2021-06-02,3,898004819696809,06195840920,NaN,nan,NATANAEL SMITH,M,1975-09-24,LUZIA DA SILVA SMITH,99,nan,,nan,nan,1,,41,412.863,83590000,ESTRADA,CORDEIROS,41,412.863,NaT,NaT,NaT,NaT,-1,9,27,1,,2021-05-21,2021-05-26,45,NATANAELSMITH45412863,NATANAELSMITH45412863,NATANAELSMITH44412863,NATANAELSMITH46412863,NATANAELSMITHLUZIADASILVASMITH,NATANAELSMITH24091975,NaN,NATANAELSMITH26052021,2,DOUTOR ULYSSES,PR,Doutor Ulysses,DOUTOR ULYSSES,"COVID-19, Biologia Molecular"
1046869,3871232,2021-05-26,2021-05-26,2,1,2021-05-26,None,2,1,2,2021-05-22,3,700006115294201,65468627949,NaN,nan,LUCY RIZENTAL GOMES,F,1929-10-23,AMANDA BRAGA RIZENTAL,99,nan,,nan,nan,1,,41,410.690,NaN,NaN,NaN,41,410.690,2021-05-18,2021-05-26,NaT,NaT,-1,9,181,1,,NaT,2021-05-26,91,LUCYRIZENTALGOMES91410690,LUCYRIZENTALGOMES91410690,LUCYRIZENTALGOMES90410690,LUCYRIZENTALGOMES92410690,LUCYRIZENTALGOMESAMANDABRAGARIZENTAL,LUCYRIZENTALGOMES23101929,NaN,LUCYRIZENTALGOMES26052021,2,CURITIBA,PR,Curitiba,CURITIBA,Teste Molecular (RT-PCR)
1002336,3735899,2021-05-18,2021-05-28,2,1,2021-05-28,31337934,2,1,2,2021-05-28,3,700804970054190,02383594920,NaN,nan,NELSON DA SILVA BONFIN,M,1933-03-06,MARIA DE JESUS BONFIN,99,nan,,nan,nan,1,,41,410.860,87360000,MARQUES DE CAMPO MOURÃO,JD CURITIBA,41,410.860,NaT,2021-05-18,2021-05-18,NaT,-1,12,167,2,,2021-05-16,2021-05-18,88,NELSONDASILVABONFIN88410860,NELSONDASILVABONFIN88410860,NELSONDASILVABONFIN87410860,NELSONDASILVABONFIN89410860,NELSONDASILVABONFINMARIADEJESUSBONFIN,NELSONDASILVABONFIN06031933,NaN,NELSONDASILVABONFIN18052021,11,GOIOERE,PR,Goioerê,GOIOERE,Teste Rápido Antígeno (Imunocromatografia)
1001394,3732954,2021-05-18,2021-05-23,2,1,2021-05-23,31331578,2,1,2,2021-05-23,3,706407147026780,84849975968,NaN,nan,JAIME DE MORAES JUNIOR,M,1974-12-25,ZAILDA SALETE CECCHIN DE MORAES,99,nan,,nan,nan,1,,41,410.940,NaN,NaN,NaN,41,410.690,NaT,NaT,NaT,NaT,-1,12,167,2,,2021-05-14,2021-05-18,46,JAIMEDEMORAESJUNIOR46410940,JAIMEDEMORAESJUNIOR46410690,JAIMEDEMORAESJUNIOR45410940,JAIMEDEMORAESJUNIOR47410940,JAIMEDEMORAESJUNIORZAILDASALETECECCHINDEMORAES,JAIMEDEMORAESJUNIOR25121974,NaN,JAIMEDEMORAESJUNIOR18052021,5,GUARAPUAVA,PR,Guarapuava,CURITIBA,Teste Rápido Antígeno (Imunocromatografia)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112996,4073056,2021-06-06,2021-06-11,2,9,2021-06-11,307812359,2,1,2,2021-06-10,3,No

In [70]:
novos_obitos = novos_obitos.loc[~novos_obitos['paciente'].isin(['JOSE ALBERICO DE LIMA','EURIDES DOMINGOS TRONQUIN'])]
novos_obitos

,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_less,hash_more,hash_mae,hash_nasc,hash_lib,hash_diag,rs,mun_resid,uf,municipio,mun_atend,exame_name
962896,3622004,2021-05-11,2021-06-02,2,10,2021-06-02,307583708,2,1,2,2021-05-24,3,702306173383614,92902049900,NaN,nan,EUGENIA AMABILE MANFRIM,F,1937-05-28,BARBERA TONELLO,99,5,APOSENTADO,nan,nan,1,,41,412.250,87320000,RONCADOR,KM 35 // SITIO MANFRIM,41,411.520,2021-05-04,2021-05-04,2021-05-04,2021-05-04,1.605.854,9,27,1,LABORATORIO NUCLEO DIAGNOSTICO,2021-05-01,2021-05-04,83,EUGENIAAMABILEMANFRIM83412250,EUGENIAAMABILEMANFRIM83411520,EUGENIAAMABILEMANFRIM82412250,EUGENIAAMABILEMANFRIM84412250,EUGENIAAMABILEMANFRIMBARBERATONELLO,EUGENIAAMABILEMANFRIM28051937,EUGENIAAMABILEMANFRIM04052021,EUGENIAAMABILEMANFRIM04052021,11,RONCADOR,PR,Roncador,MARINGA,"COVID-19, Biologia Molecular"
1050155,3881279,2021-05-26,NaT,1,1,2021-06-02,None,2,1,2,2021-06-02,3,898004819696809,06195840920,NaN,nan,NATANAEL SMITH,M,1975-09-24,LUZIA DA SILVA SMITH,99,nan,,nan,nan,1,,41,412.863,83590000,ESTRADA,CORDEIROS,41,412.863,NaT,NaT,NaT,NaT,-1,9,27,1,,2021-05-21,2021-05-26,45,NATANAELSMITH45412863,NATANAELSMITH45412863,NATANAELSMITH44412863,NATANAELSMITH46412863,NATANAELSMITHLUZIADASILVASMITH,NATANAELSMITH24091975,NaN,NATANAELSMITH26052021,2,DOUTOR ULYSSES,PR,Doutor Ulysses,DOUTOR ULYSSES,"COVID-19, Biologia Molecular"
1046869,3871232,2021-05-26,2021-05-26,2,1,2021-05-26,None,2,1,2,2021-05-22,3,700006115294201,65468627949,NaN,nan,LUCY RIZENTAL GOMES,F,1929-10-23,AMANDA BRAGA RIZENTAL,99,nan,,nan,nan,1,,41,410.690,NaN,NaN,NaN,41,410.690,2021-05-18,2021-05-26,NaT,NaT,-1,9,181,1,,NaT,2021-05-26,91,LUCYRIZENTALGOMES91410690,LUCYRIZENTALGOMES91410690,LUCYRIZENTALGOMES90410690,LUCYRIZENTALGOMES92410690,LUCYRIZENTALGOMESAMANDABRAGARIZENTAL,LUCYRIZENTALGOMES23101929,NaN,LUCYRIZENTALGOMES26052021,2,CURITIBA,PR,Curitiba,CURITIBA,Teste Molecular (RT-PCR)
1002336,3735899,2021-05-18,2021-05-28,2,1,2021-05-28,31337934,2,1,2,2021-05-28,3,700804970054190,02383594920,NaN,nan,NELSON DA SILVA BONFIN,M,1933-03-06,MARIA DE JESUS BONFIN,99,nan,,nan,nan,1,,41,410.860,87360000,MARQUES DE CAMPO MOURÃO,JD CURITIBA,41,410.860,NaT,2021-05-18,2021-05-18,NaT,-1,12,167,2,,2021-05-16,2021-05-18,88,NELSONDASILVABONFIN88410860,NELSONDASILVABONFIN88410860,NELSONDASILVABONFIN87410860,NELSONDASILVABONFIN89410860,NELSONDASILVABONFINMARIADEJESUSBONFIN,NELSONDASILVABONFIN06031933,NaN,NELSONDASILVABONFIN18052021,11,GOIOERE,PR,Goioerê,GOIOERE,Teste Rápido Antígeno (Imunocromatografia)
1001394,3732954,2021-05-18,2021-05-23,2,1,2021-05-23,31331578,2,1,2,2021-05-23,3,706407147026780,84849975968,NaN,nan,JAIME DE MORAES JUNIOR,M,1974-12-25,ZAILDA SALETE CECCHIN DE MORAES,99,nan,,nan,nan,1,,41,410.940,NaN,NaN,NaN,41,410.690,NaT,NaT,NaT,NaT,-1,12,167,2,,2021-05-14,2021-05-18,46,JAIMEDEMORAESJUNIOR46410940,JAIMEDEMORAESJUNIOR46410690,JAIMEDEMORAESJUNIOR45410940,JAIMEDEMORAESJUNIOR47410940,JAIMEDEMORAESJUNIORZAILDASALETECECCHINDEMORAES,JAIMEDEMORAESJUNIOR25121974,NaN,JAIMEDEMORAESJUNIOR18052021,5,GUARAPUAVA,PR,Guarapuava,CURITIBA,Teste Rápido Antígeno (Imunocromatografia)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112996,4073056,2021-06-06,2021-06-11,2,9,2021-06-11,307812359,2,1,2,2021-06-10,3,No

In [71]:
novos_obitos[['id','paciente','sexo','idade','mun_resid', 'rs', 'data_cura_obito','hash']].to_excel(join(default_output,'novos_obitos.xlsx'))

In [72]:
novos_10dias = novos_casos.loc[novos_casos['data_diagnostico']>=today-timedelta(10)]
retroativos = novos_casos.loc[novos_casos['data_diagnostico']<today-timedelta(10)]

novos_10dias_sample = novos_10dias.loc[novos_10dias['id'].isin(novos_obitos['id'])]
retroativos_sample = retroativos.loc[retroativos['id'].isin(novos_obitos['id'])]

retroativos = retroativos.loc[~retroativos['id'].isin(novos_obitos['id'])]
novos_10dias = novos_10dias.loc[~novos_10dias['id'].isin(novos_obitos['id'])]

retroativos_sample = pd.concat([retroativos_sample,retroativos.sample(n=1219,random_state=1)])
novos_10dias_sample = pd.concat([novos_10dias_sample,novos_10dias.sample(n=9259,random_state=1)])

novos_casos = novos_casos.drop(index=pd.concat([retroativos_sample,novos_10dias_sample]).index)
novos_casos.to_pickle('casos_pra_entrar.pkl')

novos_casos = pd.concat([retroativos_sample,novos_10dias_sample])
novos_casos[['id','paciente','sexo','idade','mun_resid', 'mun_atend', 'rs', 'exame_name','data_diagnostico','data_com','data_1o_sintomas','hash']].to_excel(join(default_output,'novos_casos.xlsx'))
novos_casos

,id,data_notificacao,data_encerramento,status_notificacao,origem,updated_at,numero_do,classificacao_final,criterio_classificacao,evolucao,data_cura_obito,tipo_paciente,cns,cpf,passaporte,etnia,paciente,sexo,data_nascimento,nome_mae,raca_cor,ocupacao,ocupacao_descricao,cod_cbo,escolaridade,pais_residencia,pais_municipio_residencia,uf_residencia,ibge_residencia,cep_residencia,logradouro_residencia,bairro_residencia,uf_unidade_notifica,ibge_unidade_notifica,data_cadastro,data_coleta,data_recebimento,data_liberacao,requisicao,resultado,exame,metodo,lab_executor,data_1o_sintomas,data_diagnostico,idade,hash,hash_atend,hash_less,hash_more,hash_mae,hash_nasc,hash_lib,hash_diag,rs,mun_resid,uf,municipio,mun_atend,exame_name,data_com
1174102,165377,2020-05-29,2021-06-17,2,2,2021-06-17,30743290,2,1,2,2021-05-21,3,704600670789129,74873164915,NaN,nan,LURDES FATIMA HALABURA,F,1966-08-14,JULIA WUDARSKI HALABURA,1,nan,Não Informado,nan,nan,1,,41,410.690,81925620,GABRIEL RIBEIRO,SITIO CERCADO,41,410.690,NaT,NaT,NaT,NaT,210.206.156.629,9,27,1,,2020-05-27,2020-05-29,53,LURDESFATIMAHALABURA53410690,LURDESFATIMAHALABURA53410690,NaN,NaN,LURDESFATIMAHALABURAJULIAWUDARSKIHALABURA,LURDESFATIMAHALABURA14081966,NaN,LURDESFATIMAHALABURA29052020,2,CURITIBA,PR,Curitiba,CURITIBA,"COVID-19, Biologia Molecular",2021-06-18
1176228,4289708,2021-06-17,2020-09-12,2,1,2021-06-17,None,2,1,2,2020-08-12,3,702305104407010,96222832920,NaN,nan,ODETE DAS GRACAS CRITCHII,F,1951-03-12,SEBASTIANA ALVES FARIAS,99,nan,,nan,nan,1,,41,412.760,NaN,NaN,NaN,41,410.690,2020-07-23,2020-07-23,2020-07-23,2020-07-23,200.023.005.240,9,27,1,GENOPRIMER DIAGNOSTICO MOLECULAR - CURITIBA,2020-07-20,2020-07-23,70,ODETEDASGRACASCRITCHII70412760,ODETEDASGRACASCRITCHII70410690,NaN,NaN,ODETEDASGRACASCRITCHIISEBASTIANAALVESFARIAS,ODETEDASGRACASCRITCHII12031951,ODETEDASGRACASCRITCHII23072020,ODETEDASGRACASCRITCHII23072020,2,TIJUCAS DO SUL,PR,Tijucas do Sul,CURITIBA,"COVID-19, Biologia Molecular",2021-06-18
1166120,2306461,2021-02-07,2021-06-15,2,1,2021-06-16,31832112,2,1,2,2021-06-11,3,708407741331562,06341741936,NaN,nan,ROBSON RODRIGO DO AMARAL,M,1990-11-18,REGINA APARECIDA PEREIRA DO AMARAL,1,5,MOTORISTA,5.199,nan,1,,41,411.370,86036370,Estrada dos Pioneiros,Jardim Morumbi,41,999.999,2021-02-05,2021-02-05,2021-02-06,2021-02-07,211.703.020.015,9,27,1,UEL -UNIVERSIDADE ESTADUAL DE LONDRINA,2021-05-30,2021-02-05,30,ROBSONRODRIGODOAMARAL30411370,ROBSONRODRIGODOAMARAL30999999,NaN,NaN,ROBSONRODRIGODOAMARALREGINAAPARECIDAPEREIRADOA...,ROBSONRODRIGODOAMARAL18111990,ROBSONRODRIGODOAMARAL07022021,ROBSONRODRIGODOAMARAL05022021,17,LONDRINA,PR,Londrina,IGNORADO/EXTERIOR/IG,"COVID-19, Biologia Molecular",2021-06-18
1174270,2911601,2021-03-17,2021-06-17,2,1,2021-06-17,31836205,2,1,2,2021-05-29,3,705004463329553,05270761980,NaN,nan,DOUGLAS MOREIRA FERRAZ,M,1986-06-01,EBER JOSE DE PAULA FERRAZ,99,nan,,nan,nan,1,,41,410.580,83405680,Rua Guilherme Rodbard,Campo Pequeno,41,410.420,2021-03-19,2021-03-19,2021-03-19,2021-03-20,210.207.099.556,9,1,1,LABORATORIO CENTRAL DO ESTADO,2021-03-12,2021-03-19,34,DOUGLASMOREIRAFERRAZ34410580,DOUGLASMOREIRAFERRAZ34410420,NaN,NaN,DOUGLASMOREIRAFERRAZEBERJOSEDEPAULAFERRAZ,DOUGLASMOREIRAFERRAZ01061986,DOUGLASMOREIRAFERRAZ20032021,DOUGLASMOREIRAFERRAZ19032021,2,COLOMBO,PR,Colombo,CAMPO LARGO,Vírus Respiratório,2021-06-18
1178890,2668405,2021-03-05,2021-06-18,2,2,2021-06-18,30744850,2,1,2,2021-05-21,3,708406790969760,14215640920,NaN,nan,RUBENS NELSON GRITA,M,1950-09-23,ELIZA GRITA,1,nan,,nan,nan,1,,41,410.690,82710150,ALBERTO OTTO,SANTA CANDIDA,41,410.690,2021-04-12,2021-04-12,2021-04-12,2021-04-12,210.206.142.366,9,27,1,INSTITUTO DE BIOLOGIA MOLECULAR DO PARANA - IBMP,2021-03-03,2021-04-12,70,RUBENSNELSONGRITA70410690,RUBENSNELSONGRITA70410690,NaN,NaN,RUBENSNELSONGRITAELIZAGRITA,RUBENSNELSONGRITA23091950,RUBENSNELSONGRITA12042021,RUBENSNELSONGRITA12042021,2,CURITIBA,PR,Curitiba,CURITIBA,"COVID-19, Biologia Molecular",2021-06-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [73]:
date_parser = lambda x: x
relatorio_file = join(default_output,f"relatorio_{(datetime.today().strftime('%d/%m/%Y_%Hh').replace('/','_').replace(' ',''))}.txt")
relatorio_file

'C:\\sesa\\dev\\output\\relatorio_18_06_2021_13h.txt'

In [74]:
casos_confirmadosPR = casos_confirmados.loc[casos_confirmados['rs']!=99]

obitos_confirmados =  casos_confirmados.loc[casos_confirmados['evolucao']==2]

obitos_confirmadosPR = obitos_confirmados.loc[obitos_confirmados['rs']!=99]

print(f"Total de casos: {len(casos_confirmados)} + {len(novos_casos)}")
print(f"Total de obitos: {len(obitos_confirmados)} + {len(novos_obitos)}\n\n")

novos_casosPR = novos_casos.loc[novos_casos['rs']!=99].copy()
print(f"Total de casos PR: {len(casos_confirmadosPR)} + {len(novos_casosPR)}")

novos_obitosPR = novos_obitos.loc[novos_obitos['rs']!=99].copy()
print(f"Total de obitos PR: {len(obitos_confirmadosPR)} + {len(novos_obitosPR)}")

novos_casosFora = novos_casos.loc[novos_casos['rs']==99].copy()
print(f"Total de casos Fora: {len(casos_confirmados) - len(casos_confirmadosPR)} + {len(novos_casosFora)}")

novos_obitosFora = novos_obitos.loc[novos_obitos['rs']==99].copy()
print(f"Total de obitos Fora: {len(obitos_confirmados) - len(obitos_confirmadosPR)} + {len(novos_obitosFora)}")


novos_obitosPR_group = novos_obitosPR.groupby(by='municipio')

data_retroativos = ontem - timedelta(14)

retroativos = novos_casosPR.loc[(novos_casosPR['data_diagnostico'].apply(date_parser) <= data_retroativos)].sort_values(by='data_diagnostico')
last2weeks = novos_casosPR.loc[(novos_casosPR['data_diagnostico'].apply(date_parser) > data_retroativos) & (novos_casosPR['data_diagnostico'].apply(date_parser) <= anteontem)].sort_values(by='data_diagnostico')
hoje = novos_casosPR.loc[(novos_casosPR['data_diagnostico'].apply(date_parser) > anteontem)].sort_values(by='data_diagnostico')

obitos_retroativos = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) <= data_retroativos)].sort_values(by='data_cura_obito')
obitos_last2weeks = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > data_retroativos) & (novos_obitosPR['data_cura_obito'].apply(date_parser) <= anteontem)].sort_values(by='data_cura_obito')
obitos_hoje = novos_obitosPR.loc[(novos_obitosPR['data_cura_obito'].apply(date_parser) > anteontem)].sort_values(by='data_cura_obito')

# relatorio_file = join(default_output,f"relatorio_{(today.strftime('%d/%m/%Y_%Hh').replace('/','_').replace(' ',''))}.txt")
with codecs.open(relatorio_file,"w","utf-8-sig") as relatorio:
    relatorio.write(f"{today.strftime('%d/%m/%Y')}\n")
    relatorio.write(f"{len(novos_casosPR):,} novos casos residentes ".replace(',','.'))

    if len(novos_casosFora) > 0:
        relatorio.write(f"e {len(novos_casosFora):,} não residente{'s' if len(novos_casosFora) > 1 else ''} ".replace(',','.'))
    relatorio.write(f"divulgados no PR.\n")

    relatorio.write(f"{len(casos_confirmadosPR)+len(novos_casosPR):,} casos confirmados residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(casos_confirmados)+len(novos_casos):,} total geral.\n\n".replace(',','.'))
    relatorio.write(f"{len(novos_obitosPR):,} Óbitos residentes do PR:\n".replace(',','.'))

    for municipio, obitos in novos_obitosPR_group:
        relatorio.write(f"{len(obitos):,} {municipio}\n".replace(',','.'))

    if len(novos_obitosFora) > 0:
        relatorio.write('\n')
        relatorio.write(f"{len(novos_obitosFora):,} Óbito{'s' if len(novos_obitosFora) > 1 else ''} não residente{'s' if len(novos_obitosFora) > 1 else ''} do PR.\n".replace(',','.'))

    relatorio.write('\n')
    relatorio.write(f"{len(obitos_confirmadosPR)+len(novos_obitosPR):,} óbitos residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(obitos_confirmados)+len(novos_obitos):,} total geral.\n\n".replace(',','.'))

    for _, row in novos_obitos.iterrows():
       relatorio.write(f"{row['sexo']}\t{int(row['idade'])}\t{row['municipio'] if row['rs']!=99 else (row['municipio']+'/'+row['uf'])}\t{str(int(row['rs'])).zfill(2) if row['rs']!=99 else '#N/D'}\t{row['data_cura_obito'].day}/{static.meses[row['data_cura_obito'].month-1]}/{row['data_cura_obito'].year}\n")
    relatorio.write('\n')

    if True:

        #casos
        relatorio.write(f"{len(novos_casosPR):,} novos casos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(retroativos):,} casos retroativos confirmados no período de {retroativos.iloc[0]['data_diagnostico'].strftime('%d/%m/%Y')} à {retroativos.iloc[-1]['data_diagnostico'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(last2weeks):,} novos casos confirmados no período de {last2weeks.iloc[0]['data_diagnostico'].strftime('%d/%m/%Y')} à {last2weeks.iloc[-1]['data_diagnostico'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(hoje):,} novos casos confirmados hoje.\n\n".replace(',','.'))


        novos_casosPR['month'] = novos_casosPR.apply(lambda x: x['data_diagnostico'].month, axis=1)
        novos_casosPR['year'] = novos_casosPR.apply(lambda x: x['data_diagnostico'].year, axis=1)
        relatorio.write('Novos casos por meses:\n')
        for group, value in novos_casosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')
        
        relatorio.write('Novos casos por dia:\n')
        for group, value in novos_casosPR.groupby(by='data_diagnostico'):
            relatorio.write(f"{group.strftime('%d/%m/%Y')}\t{len(value)}\n")
        relatorio.write('\n')

        #obitos
        relatorio.write(f"{len(novos_obitosPR):,} novos obitos residentes divulgados no PR.\n".replace(',','.'))

        relatorio.write(f"{len(obitos_retroativos):,} obitos retroativos ocorridos no período de {obitos_retroativos.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_retroativos.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(obitos_last2weeks):,} novos obitos ocorridos no período de {obitos_last2weeks.iloc[0]['data_cura_obito'].strftime('%d/%m/%Y')} à {obitos_last2weeks.iloc[-1]['data_cura_obito'].strftime('%d/%m/%Y')}.\n".replace(',','.'))
        relatorio.write(f"{len(obitos_hoje):,} novos obitos ocorridos hoje.\n\n".replace(',','.'))

        relatorio.write('Novos obitos por meses:\n')
        novos_obitosPR['month'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].month, axis=1)
        novos_obitosPR['year'] = novos_obitosPR.apply(lambda x: x['data_cura_obito'].year, axis=1)
        for group, value in novos_obitosPR.groupby(by=['year','month']):
            relatorio.write(f"{static.meses[int(group[1])-1]}\\{group[0]}: {len(value)}\n")
        relatorio.write('\n')

        relatorio.write('Novos obitos por dia:\n')
        for group, value in novos_obitosPR.groupby(by='data_cura_obito'):
            relatorio.write(f"{group.strftime('%d/%m/%Y')}\t{len(value)}\n")
        relatorio.write('\n')

        #-----RELATÓRIO DA COMUNICAÇÃO--------------
        obitos_list = []
        munic = []
        for municipio, obitos in novos_obitosPR_group:
            obito = len(obitos)
            obitos_list.append(obito)
            munic.append(municipio)             

        dicionario = (dict(zip(list(munic),list(obitos_list))))
        #print(dicionario)
        dicionario = sorted(dicionario.items(),key=lambda x: x[1], reverse = True)
        #print(dicionario)


        relatorio.write(f"\nOs pacientes que foram a óbito residiam em: ")
        for municip, obit in dict(dicionario).items():
            if obit != 1:
                relatorio.write(f"{municip} ({obit})")
                relatorio.write(f", ")
        relatorio.write(f".\n")
        relatorio.write(f"A Sesa registra ainda a morte de uma pessoa que residia em cada um dos seguintes municípios:")
        for municip, obit in dict(dicionario).items():
            if obit == 1:
                relatorio.write(f"{municip}")
                relatorio.write(f", ")


Total de casos: 1170632 + 10807
Total de obitos: 29177 + 497


Total de casos PR: 1164281 + 10751
Total de obitos PR: 29013 + 495
Total de casos Fora: 6351 + 56
Total de obitos Fora: 164 + 2


In [75]:
with codecs.open(relatorio_file,"r","utf-8-sig") as relatorio:
    print(relatorio.read())

18/06/2021
10.751 novos casos residentes e 56 não residentes divulgados no PR.
1.175.032 casos confirmados residentes do PR.
1.181.439 total geral.

495 Óbitos residentes do PR:
3 Almirante Tamandaré
1 Alto Paraná
1 Alto Paraíso
1 Altônia
1 Antonina
5 Apucarana
12 Arapongas
1 Assaí
1 Assis Chateaubriand
1 Astorga
2 Barracão
1 Borrazópolis
2 Cambira
1 Cambé
1 Campina Grande do Sul
4 Campina da Lagoa
1 Campo Bonito
10 Campo Mourão
2 Candói
2 Carlópolis
8 Cascavel
2 Castro
3 Cerro Azul
2 Cianorte
2 Cidade Gaúcha
2 Clevelândia
8 Colombo
1 Colorado
2 Contenda
2 Cornélio Procópio
2 Coronel Vivida
1 Cruz Machado
2 Cruzeiro do Oeste
81 Curitiba
1 Curiúva
1 Cândido de Abreu
1 Diamante do Norte
1 Doutor Ulysses
3 Engenheiro Beltrão
6 Faxinal
2 Fazenda Rio Grande
1 Flor da Serra do Sul
1 Floraí
28 Foz do Iguaçu
1 Foz do Jordão
3 Francisco Beltrão
1 Fênix
4 General Carneiro
5 Goioerê
1 Grandes Rios
12 Guarapuava
1 Guaratuba
1 Ibaiti
2 Irati
1 Itaipulândia
1 Itapejara d'Oeste
2 Ivaiporã
2 Jandaia d

In [76]:
cc = CasosConfirmados()
cc.load(f"cc_{ontem.strftime('%d_%m_%Y')}")
casos_confirmados = cc.df
print(len(casos_confirmados))
casos_confirmados[['identificacao','evolucao']].groupby('evolucao').count()

databases: ['cc_11_06_2021', 'cc_15_06_2021', 'cc_16_06_2021', 'cc_17_06_2021', 'cc_18_06_2021']
Load C:\sesa\dev\bulletin\database\casos_confirmados\cc_17_06_2021.pkl
loading Casos Confirmados from pkl time elepsed:00:00:03
1170632


,identificacao
evolucao,
1,824521
2,29177
3,316934


In [77]:
novos_casos = novos_casos.rename(columns={
    'id':'id_notifica',
    'ibge_residencia': 'ibge_resid',
    'ibge_unidade_notifica': 'ibge_atend',
    'paciente':'nome',
    'sexo': 'sexo',
    'idade':'idade',
    'exame': 'laboratorio',
    'data_diagnostico':'dt_diag',
    'data_com':'comunicacao',
    'data_1o_sintomas': 'is',
    'evolucao': 'evolucao',
    'data_cura_obito': 'data_evolucao',
    'uf': 'uf_resid'
})

novos_casos = novos_casos[['id_notifica', 'uf_resid', 'ibge_resid', 'ibge_atend', 'nome', 'sexo', 'idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao', 'data_evolucao']]

novos_casos.loc[novos_casos['evolucao']==5,'evolucao'] = 2
novos_casos.loc[~novos_casos['evolucao'].isin([1,2,3]),'evolucao'] = 3
novos_casos.loc[novos_casos['evolucao']==2,'evolucao'] = 3

novos_casos['hash'] = (novos_casos['nome'].apply(normalize_hash) +
                  novos_casos['idade'].astype(int).astype(str) +
                  novos_casos['ibge_resid'].astype(int).astype(str) )

novos_casos['hash_less'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x - 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_more'] = ( novos_casos['nome'].apply(normalize_hash) +
                        novos_casos['idade'].apply(lambda x: str(x + 1)) +
                        novos_casos['ibge_resid'].astype(str) )

novos_casos['hash_atend'] = (novos_casos['nome'].apply(normalize_hash) +
                    novos_casos['idade'].astype(str) +
                    novos_casos['ibge_atend'].apply(str) )

novos_casos['hash_less_atend'] = (novos_casos['nome'].apply(normalize_hash) +
                    novos_casos['idade'].apply(lambda x: str(x-1)) +
                    novos_casos['ibge_atend'].apply(str) )

novos_casos['hash_more_atend'] = (novos_casos['nome'].apply(normalize_hash) +
                    novos_casos['idade'].apply(lambda x: str(x+1)) +
                    novos_casos['ibge_atend'].apply(str) )

novos_casos['identificacao'] = list(range(casos_confirmados.iloc[-1,0] + 1,casos_confirmados.iloc[-1,0] + 1 + len(novos_casos)))

casos_confirmados = casos_confirmados.append(novos_casos)
print(len(casos_confirmados))
# casos_confirmados.groupby('evolucao')[['sexo']].count()
casos_confirmados[['identificacao','evolucao']].groupby('evolucao').count()

1181439


,identificacao
evolucao,
1,825482
2,29177
3,326780


In [78]:
casos_confirmados.loc[casos_confirmados['hash'].isin(novos_obitos['hash'])]

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
119108,175703,-1,PR,410.690,410.690,MARIA NATALINA DE OLIVEIRA ROQUE,F,59,IBMP,2020-08-24,2020-08-25,2020-08-16,1,2020-08-25,2020-09-10,MARIANATALINADEOLIVEIRAROQUE59410690,MARIANATALINADEOLIVEIRAROQUE58410690,MARIANATALINADEOLIVEIRAROQUE60410690,MARIANATALINADEOLIVEIRAROQUE59410690,MARIANATALINADEOLIVEIRAROQUE58410690,MARIANATALINADEOLIVEIRAROQUE60410690,MARIANATALINADEOLIVEIRAROQUE24082020
211063,288421,-1,PR,412.560,412.560,ADILSON ALVES DE BRITO,M,55,"COVID-19, Biologia Molecular",2020-10-31,2020-11-01,2020-10-26,3,NaT,NaT,ADILSONALVESDEBRITO55412560,ADILSONALVESDEBRITO54412560,ADILSONALVESDEBRITO56412560,ADILSONALVESDEBRITO55412560,ADILSONALVESDEBRITO54412560,ADILSONALVESDEBRITO56412560,ADILSONALVESDEBRITO31102020
397073,474962,-1,PR,410.140,410.140,MOISES DANIEL DA SILVA,M,82,Teste Rápido IgM/IgG não discriminatório (Imun...,2020-12-25,2020-12-26,NaT,1,2020-12-26,2020-12-26,MOISESDANIELDASILVA82410140,MOISESDANIELDASILVA81410140,MOISESDANIELDASILVA83410140,MOISESDANIELDASILVA82410140,MOISESDANIELDASILVA81410140,MOISESDANIELDASILVA83410140,MOISESDANIELDASILVA25122020
649872,731834,-1,PR,412.120,412.120,TEREZINHA DE JESUS DOS SANTOS MATOS,F,56,"COVID-19, Biologia Molecular",2021-02-18,2021-03-01,2021-02-12,3,NaT,NaT,TEREZINHADEJESUSDOSSANTOSMATOS56412120,TEREZINHADEJESUSDOSSANTOSMATOS55412120,TEREZINHADEJESUSDOSSANTOSMATOS57412120,TEREZINHADEJESUSDOSSANTOSMATOS56412120,TEREZINHADEJESUSDOSSANTOSMATOS55412120,TEREZINHADEJESUSDOSSANTOSMATOS57412120,TEREZINHADEJESUSDOSSANTOSMATOS18022021
663493,745461,-1,PR,412.120,412.120,JOAO NERES DA SILVA,M,66,"COVID-19, Biologia Molecular",2021-03-03,2021-03-04,2021-02-24,3,NaT,NaT,JOAONERESDASILVA66412120,JOAONERESDASILVA65412120,JOAONERESDASILVA67412120,JOAONERESDASILVA66412120,JOAONERESDASILVA65412120,JOAONERESDASILVA67412120,JOAONERESDASILVA03032021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175157,1261903,4208716,PR,410.830,410.830,LUIZ CARLOS TEIXEIRA PRESTES,M,53,27,2021-06-15,2021-06-18,2021-06-10,3,2021-06-17,NaT,LUIZCARLOSTEIXEIRAPRESTES53410830,LUIZCARLOSTEIXEIRAPRESTES52.0410830.0,LUIZCARLOSTEIXEIRAPRESTES54.0410830.0,LUIZCARLOSTEIXEIRAPRESTES53.0410830.0,LUIZCARLOSTEIXEIRAPRESTES52.0410830.0,LUIZCARLOSTEIXEIRAPRESTES54.0410830.0,NaN
1175609,1261904,4249748,PR,412.630,412.630,ROSIDETE RIBEIRO DA SILVA RAMOS,F,48,27,2021-06-15,2021-06-18,2021-06-07,3,2021-06-15,NaT,ROSIDETERIBEIRODASILVARAMOS48412630,ROSIDETERIBEIRODASILVARAMOS47.0412630.0,ROSIDETERIBEIRODASILVARAMOS49.0412630.0,ROSIDETERIBEIRODASILVARAMOS48.0412630.0,ROSIDETERIBEIRODASILVARAMOS47.0412630.0,ROSIDETERIBEIRODASILVARAMOS49.0412630.0,NaN
1170382,1261905,4270137,PR,411.370,411.370,DANILO DOMINGOS GUERRA,M,12,-1,2021-06-16,2021-06-18,2021-06-01,3,2021-06-09,NaT,DANILODOMINGOSGUERRA12411370,DANILODOMINGOSGUERRA11.0411370.0,DANILODOMINGOSGUERRA13.0411370.0,DANILODOMINGOSGUERRA12.0411370.0,DANILODOMINGOSGUERRA11.0411370.0,DANILODOMINGOSGUERRA13.0411370.0,NaN
1179456,1261906,4281957,PR,411.340,410.640,MAIKON JESUS DOS SANTOS,M,26,167,2021-06-17,2021-06-18,2021-06-08,3,2021-06-17,NaT,MAIKONJESUSDOSSANTOS26411340,MAIKONJESUSDOSSANTOS25.0411340.0,MAIKONJESUSDOSSANTOS27.0411340.0,MAIKONJESUSDOSSANTOS26.0410640.0,MAIKONJESUSDOSSANTOS25.0410640.0,MAIKONJESUSDOSSANTOS27.0410640.0,NaN


In [79]:
novos_obitos = novos_obitos.rename(columns={
    'id': 'id_notifica',
    'data_cura_obito': 'data_evolucao',
    'paciente':'nome'
})

novos_obitos['data_com_evolucao'] = pd.to_datetime(date.today())
novos_obitos['evolucao'] = 2

novos_obitos = novos_obitos[['hash','id_notifica','evolucao','data_evolucao','data_com_evolucao']]
novos_obitos = novos_obitos.set_index('hash')
novos_obitos

,id_notifica,evolucao,data_evolucao,data_com_evolucao
hash,,,,
EUGENIAAMABILEMANFRIM83412250,3622004,2,2021-05-24,2021-06-18
NATANAELSMITH45412863,3881279,2,2021-06-02,2021-06-18
LUCYRIZENTALGOMES91410690,3871232,2,2021-05-22,2021-06-18
NELSONDASILVABONFIN88410860,3735899,2,2021-05-28,2021-06-18
JAIMEDEMORAESJUNIOR46410940,3732954,2,2021-05-23,2021-06-18
...,...,...,...,...
VENILDELISBOA51410830,4073056,2,2021-06-10,2021-06-18
WAGNERJAREMCZUKDOSSANTOS27410440,4155640,2,2021-06-12,2021-06-18
WALADOMINGOSDOSSANTOS44410690,4270381,2,2021-06-14,2021-06-18


In [80]:
casos_confirmados = casos_confirmados.set_index('hash')
casos_confirmados.update(novos_obitos)
casos_confirmados.reset_index(inplace=True)

print(len(casos_confirmados))
casos_confirmados[['identificacao','evolucao']].groupby('evolucao').count()

1181439


,identificacao
evolucao,
1,825462
2,29674
3,326303


In [81]:
novos_obitos = novos_obitos.reset_index()
novos_obitos.loc[novos_obitos.duplicated('hash',keep=False)]

,hash,id_notifica,evolucao,data_evolucao,data_com_evolucao


In [82]:
casos_confirmados[['identificacao','evolucao']].groupby('evolucao').count()

,identificacao
evolucao,
1,825462
2,29674
3,326303


In [83]:
# cc.df = casos_confirmados[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend','hash_diag']]
# cc.df
casos_confirmados.loc[casos_confirmados['hash'].isin(novos_obitos.index.tolist())]

,hash,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag


In [84]:
cc.df = casos_confirmados[['identificacao','id_notifica','uf_resid','ibge_resid','ibge_atend','nome','sexo','idade','laboratorio','dt_diag','comunicacao','is','evolucao','data_evolucao','data_com_evolucao','hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend','hash_diag']]
cc.save(f"cc_{today.strftime('%d_%m_%Y')}", replace=True)
cc.df.tail()

Load C:\sesa\dev\bulletin\database\casos_confirmados\cc_18_06_2021.pkl
saving Casos Confirmados to pkl time elepsed:00:00:11


,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
1181434,1271162,4.239.566,PR,412.550,412.550,JESSICA KATHLYN BATISTA,F,21,27,2021-06-14,2021-06-18,2021-06-12,3,2021-06-14,NaT,JESSICAKATHLYNBATISTA21412550,JESSICAKATHLYNBATISTA20.0412550.0,JESSICAKATHLYNBATISTA22.0412550.0,JESSICAKATHLYNBATISTA21.0412550.0,JESSICAKATHLYNBATISTA20.0412550.0,JESSICAKATHLYNBATISTA22.0412550.0,NaN
1181435,1271163,4.186.726,PR,411.420,411.420,SUELI CINQUINI DA MATA,F,52,167,2021-06-11,2021-06-18,2021-06-07,3,NaT,NaT,SUELICINQUINIDAMATA52411420,SUELICINQUINIDAMATA51.0411420.0,SUELICINQUINIDAMATA53.0411420.0,SUELICINQUINIDAMATA52.0411420.0,SUELICINQUINIDAMATA51.0411420.0,SUELICINQUINIDAMATA53.0411420.0,NaN
1181436,1271164,4.187.750,PR,411.840,411.840,ROBERTO BATISTA DE OLIVEIRA JUNIOR,M,29,167,2021-06-09,2021-06-18,NaT,3,NaT,NaT,ROBERTOBATISTADEOLIVEIRAJUNIOR29411840,ROBERTOBATISTADEOLIVEIRAJUNIOR28.0411840.0,ROBERTOBATISTADEOLIVEIRAJUNIOR30.0411840.0,ROBERTOBATISTADEOLIVEIRAJUNIOR29.0411840.0,ROBERTOBATISTADEOLIVEIRAJUNIOR28.0411840.0,ROBERTOBATISTADEOLIVEIRAJUNIOR30.0411840.0,NaN
1181437,1271165,4.165.328,PR,411.750,411.750,ALBERTO LUCIANO PANZERI DA CRUZ,M,50,167,2021-06-10,2021-06-18,NaT,3,NaT,NaT,ALBERTOLUCIANOPANZERIDACRUZ50411750,ALBERTOLUCIANOPANZERIDACRUZ49.0411750.0,ALBERTOLUCIANOPANZERIDACRUZ51.0411750.0,ALBERTOLUCIANOPANZERIDACRUZ50.0411750.0,ALBERTOLUCIANOPANZERIDACRUZ49.0411750.0,ALBERTOLUCIANOPANZERIDACRUZ51.0411750.0,NaN
1181438,1271166,4.270.117,PR,410.580,410.580,ALEXANDRE LUIZ DE SOUZA,M,43,167,2021-06-11,2021-06-18,NaT,3,NaT,NaT,ALEXANDRELUIZDESOUZA43410580,ALEXANDRELUIZDESOUZA42.0410580.0,ALEXANDRELUIZDESOUZA44.0410580.0,ALEXANDRELUIZDESOUZA43.0410580.0,ALEXANDRELUIZDESOUZA42.0410580.0,ALEXANDRELUIZDESOUZA44.0410580.0,NaN


In [85]:
casos_confirmados = cc.df
casos_confirmados

,identificacao,id_notifica,uf_resid,ibge_resid,ibge_atend,nome,sexo,idade,laboratorio,dt_diag,comunicacao,is,evolucao,data_evolucao,data_com_evolucao,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_diag
0,8838,-1,PR,410.550,410.550,ELIANA CRISTINA CUNHA DE OLIVEIRA SOUZA,F,47,LACEN,2020-03-12,2020-03-12,NaT,1,2020-04-03,NaT,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA47410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA46410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA48410550,ELIANACRISTINACUNHADEOLIVEIRASOUZA12032020
1,10932,-1,PR,410.690,410.690,FLAMARION PETRIV DE ABREU,M,43,LACEN,2020-03-12,2020-03-12,2020-03-07,1,NaT,NaT,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU43410690,FLAMARIONPETRIVDEABREU42410690,FLAMARIONPETRIVDEABREU44410690,FLAMARIONPETRIVDEABREU12032020
2,10933,-1,PR,410.690,410.690,FLAMARION PETRIV DE ABREU JUNIOR,M,15,LACEN,2020-03-12,2020-03-12,2020-03-08,1,NaT,NaT,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR15410690,FLAMARIONPETRIVDEABREUJUNIOR14410690,FLAMARIONPETRIVDEABREUJUNIOR16410690,FLAMARIONPETRIVDEABREUJUNIOR12032020
3,11470,-1,PR,410.690,410.690,GABRIELA FUGANTI CASAGRANDE,F,26,Genoprimer,2020-03-16,2020-03-12,2020-03-11,1,NaT,NaT,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE26410690,GABRIELAFUGANTICASAGRANDE25410690,GABRIELAFUGANTICASAGRANDE27410690,GABRIELAFUGANTICASAGRANDE16032020
4,11492,-1,PR,410.690,410.690,GABRIELA SCHNEIDER GUGELMIN,F,26,Genoprimer,2020-03-18,2020-03-12,2020-03-15,1,NaT,NaT,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN26410690,GABRIELASCHNEIDERGUGELMIN25410690,GABRIELASCHNEIDERGUGELMIN27410690,GABRIELASCHNEIDERGUGELMIN18032020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181434,1271162,4.239.566,PR,412.550,412.550,JESSICA KATHLYN BATISTA,F,21,27,2021-06-14,2021-06-18,2021-06-12,3,2021-06-14,NaT,JESSICAKATHLYNBATISTA21412550,JESSICAKATHLYNBATISTA20.0412550.0,JESSICAKATHLYNBATISTA22.0412550.0,JESSICAKATHLYNBATISTA21.0412550.0,JESSICAKATHLYNBATISTA20.0412550.0,JESSICAKATHLYNBATISTA22.0412550.0,NaN
1181435,1271163,4.186.726,PR,411.420,411.420,SUELI CINQUINI DA MATA,F,52,167,2021-06-11,2021-06-18,2021-06-07,3,NaT,NaT,SUELICINQUINIDAMATA52411420,SUELICINQUINIDAMATA51.0411420.0,SUELICINQUINIDAMATA53.0411420.0,SUELICINQUINIDAMATA52.0411420.0,SUELICINQUINIDAMATA51.0411420.0,SUELICINQUINIDAMATA53.0411420.0,NaN
1181436,1271164,4.187.750,PR,411.840,411.840,ROBERTO BATISTA DE OLIVEIRA JUNIOR,M,29,167,2021-06-09,2021-06-18,NaT,3,NaT,NaT,ROBERTOBATISTADEOLIVEIRAJUNIOR29411840,ROBERTOBATISTADEOLIVEIRAJUNIOR28.0411840.0,ROBERTOBATISTADEOLIVEIRAJUNIOR30.0411840.0,ROBERTOBATISTADEOLIVEIRAJUNIOR29.0411840.0,ROBERTOBATISTADEOLIVEIRAJUNIOR28.0411840.0,ROBERTOBATISTADEOLIVEIRAJUNIOR30.0411840.0,NaN
1181437,1271165,4.165.328,PR,411.750,411.750,ALBERTO LUCIANO PANZERI DA CRUZ,M,50,167,2021-06-10,2021-06-18,NaT,3,NaT,NaT,ALBERTOLUCIANOPANZERIDACRUZ50411750,ALBERTOLUCIANOPANZERIDACRUZ49.0411750.0,ALBERTOLUCIANOPANZERIDACRUZ51.0411750.0,ALBERTOLUCIANOPANZERIDACRUZ50.0411750.0,ALBERTOLUCIANOPANZERIDACRUZ49.0411750.0,ALBERTOLUCIANOPANZERIDACRUZ51.0411750.0,NaN


In [86]:
municipios = Municipios()

macros_idx_labels = dict([ [i,x] for (i,x) in enumerate(['Leste','Oeste','Noroeste','Norte','Fora'])])
macros_labels_idx = dict([ [x,i] for (i,x) in macros_idx_labels.items() ])
municipios['macro'] = municipios['macro'].apply(lambda x: macros_labels_idx[x])

casos_confirmados = pd.merge(casos_confirmados.rename(columns={'ibge_resid':'ibge'}),municipios,on=['ibge'],how='left').rename(columns={'ibge':'ibge_resid'})

casos_confirmados['municipio_pr'] = casos_confirmados['municipio']
casos_confirmados.loc[casos_confirmados['uf_resid']!='PR','municipio_pr'] = 'Fora'

casos_confirmados.loc[casos_confirmados['uf_resid']!='PR','ibge7'] = 9999999

casos_confirmados['uf_pr'] = 'PR'
casos_confirmados.loc[casos_confirmados['uf_resid']!='PR','uf_pr'] = 'Fora'

In [87]:
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

total = casos_confirmados.groupby(['evolucao','uf_pr'])[['sexo']].count().unstack().droplevel(0,1)[['PR','Fora']]
total['GERAL'] = total.sum(1)
total = total.append(pd.Series(total.sum(0),name=0))
total.index = [2,1,3,0]
total = total.sort_index()
total = total.rename(index={0:'CASOS',2:'RECUPERADOS',1:'OBITOS',3:'ATIVOS'})
total.index.name = None
total.columns.name = None

total.astype(float)

,PR,Fora,GERAL
CASOS,1.175.032,6.407,1.181.439
OBITOS,29.508,166,29.674
RECUPERADOS,823.101,2.361,825.462
ATIVOS,322.423,3.880,326.303


In [88]:
total_macros = casos_confirmados.groupby(['macro','evolucao'])[['sexo']].count().unstack()
total_macros.columns = total_macros.columns.droplevel(0)

total_macros['CASOS'] = total_macros.sum(1)
total_macros = total_macros.fillna(0)
total_macros = total_macros.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_macros = total_macros[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_macros.index = [ macros_idx_labels[x] if isinstance(x,int) else x for x in total_macros.index]

total_macros.astype(float)

evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
Leste,542.979,14.864,361.656,166.459
Oeste,240.926,4.982,207.175,28.769
Noroeste,192.280,4.279,143.379,44.622
Norte,198.847,5.383,110.891,82.573
Fora,6.407,166,2.361,3.880


In [89]:
total_regionais = casos_confirmados.groupby(['rs','regional','evolucao'])[['sexo']].count().unstack()
total_regionais.columns = total_regionais.columns.droplevel(0)

total_regionais['CASOS'] = total_regionais.sum(1)
total_regionais = total_regionais.fillna(0)
total_regionais = total_regionais.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_regionais = total_regionais[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_regionais.astype(float)

,evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
rs,regional,,,,
1,Paranaguá,36.415,925,19.087,16.403
2,Metropolitana,344.384,10.236,239.167,94.981
3,Ponta Grossa,69.628,1.687,45.926,22.015
4,Irati,17.004,355,12.836,3.813
5,Guarapuava,38.033,831,22.534,14.668
6,União da Vitória,12.731,267,9.137,3.327
7,Pato Branco,31.855,755,27.218,3.882
8,Francisco Beltrão,43.075,738,36.903,5.434
9,Foz do Iguaçu,57.457,1.331,44.958,11.168


In [90]:
total_municipios = casos_confirmados.groupby(['macro','rs','regional','ibge7','municipio_pr','evolucao'])[['sexo']].count().unstack()

total_municipios[('','CASOS')] = total_municipios.sum(1)
total_municipios = total_municipios.fillna(0).astype(float)
total_municipios.columns = total_municipios.columns.droplevel(0)
total_municipios = total_municipios.rename(columns={1:'RECUPERADOS',2:'OBITOS',3:'ATIVOS'})
total_municipios = total_municipios[['CASOS','OBITOS','RECUPERADOS','ATIVOS']]

total_municipios.index = total_municipios.index.set_levels([ macros_idx_labels[x] if isinstance(x,int) else x for x in total_municipios.index.levels[0]], level=0)

total_municipios

evolucao                                         CASOS  OBITOS  RECUPERADOS  \
macro rs regional  ibge7   municipio_pr                                       
Leste 1  Paranaguá 4101200 Antonina              2.422      62        2.188   
                   4109500 Guaraqueçaba            648      18          548   
                   4109609 Guaratuba             4.422     161        3.952   
                   4115705 Matinhos              3.034     111        2.635   
                   4116208 Morretes              2.049      52        1.633   
...                                                ...     ...          ...   
Norte 22 Ivaiporã  4122172 Rio Branco do Ivaí      290      11          263   
                   4122651 Rosário do Ivaí         381       2          376   
                   4123857 Santa Maria do Oeste    701      23          620   
                   4125001 São João do Ivaí      1.012      20          855   
Fora  99 Fora      9999999 Fora                  6.407     166        2.361   

evolucao                                         ATIVOS  
macro rs regional  ibge7   municipio_pr                  
Leste 1  Paranaguá 4101200 Antonina                 172  
                   4109500 Guaraqueçaba              82  
                   4109609 Guaratuba                309  
                   4115705 Matinhos                 288  
                   4116208 Morretes                 364  
...                                                 ...  
Norte 22 Ivaiporã  4122172 Rio Branco do Ivaí        16  
                   4122651 Rosário do Ivaí            3  
                   4123857 Santa Maria do Oeste      58  
                   4125001 São João do Ivaí         137  
Fora  99 Fora      9999999 Fora                   3.880  

[400 rows x 4 columns]

In [100]:
total_municipios.loc[(slice(None),slice(None),slice(None),slice(None),'Curitiba')]

,,,evolucao,CASOS,OBITOS,RECUPERADOS,ATIVOS
macro,rs,regional,ibge7,,,,
Leste,2,Metropolitana,4106902,165.186,5.629,108.735,50.822


In [91]:
ministerio_cols = ['ibge7','municipio_pr','macro','CASOS','OBITOS']

ministerio_sort = pd.read_csv(join(default_input,'ministerio_ibge_order.csv'), sep=';')

ministerio = pd.merge(ministerio_sort, total_municipios.reset_index()[ministerio_cols], how='left', on='ibge7' )
del ministerio['ibge7']

ministerio['macro'] = ministerio['macro'].str.upper()

ministerio.loc[399,['municipio_pr']] = 'Fora do Parana'
ministerio.loc[399,['macro']] = ''

ministerio.loc[400,'municipio_pr'] = 'TOTAL GERAL'
ministerio.loc[400,'macro'] = ''
ministerio.loc[400,'CASOS'] = total.loc['CASOS','GERAL']
ministerio.loc[400,'OBITOS'] = total.loc['OBITOS','GERAL']

ministerio.columns = ['Município de residência', 'macro', 'Casos confirmados', 'Óbitos']

writer = pd.ExcelWriter(join(default_output,"Casos confirmados covid19 no PR por município.xlsx"),
                    engine='xlsxwriter')

ministerio.to_excel(writer,'total_municipios', index=False)
fit_cols(writer,ministerio,'total_municipios')

writer.save()

from openpyxl import load_workbook
from openpyxl.styles import Alignment, PatternFill

wb = load_workbook(filename = join(default_output,"Casos confirmados covid19 no PR por município.xlsx"))
sheet = wb['total_municipios']
sheet.insert_rows(1)
sheet.merge_cells(start_row=1, start_column=1, end_row=1, end_column=4)
cell = sheet.cell(row=1, column=1)  
cell.value = 'Casos confirmados covid-19 no Paraná'  
cell.alignment = Alignment(horizontal='center', vertical='center')  
wb.save(filename = join(default_output,"Casos confirmados covid19 no PR por município.xlsx"))

ministerio

,Município de residência,macro,Casos confirmados,Óbitos
0,Antonina,LESTE,2.422,62
1,Guaraqueçaba,LESTE,648,18
2,Guaratuba,LESTE,4.422,161
3,Matinhos,LESTE,3.034,111
4,Morretes,LESTE,2.049,52
...,...,...,...,...
396,Rosário do Ivaí,NORTE,381,2
397,Santa Maria do Oeste,NORTE,701,23
398,São João do Ivaí,NORTE,1.012,20
399,Fora do Parana,,6.407,166


In [92]:
casos_confirmados['regional'] = casos_confirmados['rs'].apply(lambda x: str(x).zfill(2)) +'ª '+ casos_confirmados['regional']
casos_confirmados = casos_confirmados[['id_notifica','macro','regional','ibge_resid','municipio','ibge_atend','nome','sexo','idade', 'laboratorio', 'dt_diag', 'comunicacao', 'is', 'evolucao','data_evolucao', 'data_com_evolucao']].copy()
casos_confirmados.loc[casos_confirmados['evolucao']==1,'evolucao'] = 'CURA'
casos_confirmados.loc[casos_confirmados['evolucao']==2,'evolucao'] = 'OBITO'
casos_confirmados.loc[casos_confirmados['evolucao']==3,'evolucao'] = 'ATIVO'

In [93]:
obitos_confirmados = casos_confirmados.loc[casos_confirmados['evolucao']=='OBITO']

In [94]:
writer = pd.ExcelWriter(join(default_output, f"casos_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx, macro in macros_idx_labels.items():
    casos_macro = casos_confirmados.loc[casos_confirmados['macro']==idx].copy()
    casos_macro['macro'] = macro
    casos_macro.to_excel(writer,macro)
    fit_cols(writer,casos_macro,macro)


writer.save()

In [95]:
writer = pd.ExcelWriter(join(default_output, f"obitos_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for idx, macro in macros_idx_labels.items():
    casos_macro = obitos_confirmados.loc[obitos_confirmados['macro']==idx].copy()
    casos_macro['macro'] = macro
    casos_macro.to_excel(writer,macro)
    fit_cols(writer,casos_macro,macro)


writer.save()

In [96]:
# casos_confirmados['rs'].unique()

In [97]:
writer = pd.ExcelWriter(join(default_output,f"casos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(casos_confirmados['regional'].unique()):
    casos_regional = casos_confirmados.loc[casos_confirmados['regional']==regional].copy()
    casos_regional['macro'] = casos_regional['macro'].apply(lambda x: macros_idx_labels[x])
    casos_regional.to_excel(writer,regional)
    fit_cols(writer,casos_regional,regional)


writer.save()

In [98]:
writer = pd.ExcelWriter(join(default_output,f"obitos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"),
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

total.to_excel(writer,'total')
total_macros.to_excel(writer,'total_macros')
total_regionais.to_excel(writer,'total_regionais')
total_municipios.to_excel(writer,'total_municipios')

fit_cols(writer,total,'total')
fit_cols(writer,total_macros,'total_macros')
fit_cols(writer,total_regionais,'total_regionais')
fit_cols(writer,total_municipios,'total_municipios')

for regional in sorted(obitos_confirmados['regional'].unique()):
    obitos_regional = obitos_confirmados.loc[obitos_confirmados['regional']==regional].copy()
    obitos_regional['macro'] = obitos_regional['macro'].apply(lambda x: macros_idx_labels[x])
    obitos_regional.to_excel(writer,regional)
    fit_cols(writer,obitos_regional,regional)


writer.save()

In [99]:
# REMOVE ARQUIVOS DE ONTEM
try:
    remove(join(default_output, f"casos_{ontem.strftime('%d_%m_%Y')}.xlsx"))
    remove(join(default_output, f"obitos_{ontem.strftime('%d_%m_%Y')}.xlsx"))
    remove(join(default_output, f"casos_regionais_{ontem.strftime('%d_%m_%Y')}.xlsx"))
    remove(join(default_output, f"obitos_regionais_{ontem.strftime('%d_%m_%Y')}.xlsx"))

    remove(join(default_output, f"casos_{ontem.strftime('%d_%m_%Y')}.zip"))
    remove(join(default_output, f"obitos_{ontem.strftime('%d_%m_%Y')}.zip"))
    remove(join(default_output, f"casos_regionais_{ontem.strftime('%d_%m_%Y')}.zip"))
    remove(join(default_output, f"obitos_regionais_{ontem.strftime('%d_%m_%Y')}.zip"))
except:
    pass

# GERA ARQUIVOS COMPACTADOS DE HOJE

paths = [join(default_output, f"casos_{today.strftime('%d_%m_%Y')}.xlsx")]
paths.append(join(default_output, f"obitos_{today.strftime('%d_%m_%Y')}.xlsx"))
paths.append(join(default_output, f"casos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"))
paths.append(join(default_output, f"obitos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"))

for path in paths:
    pz.compress(path, None, join(default_output, f"{basename(path)}.zip".replace(".xlsx", "")), "Jt\j+ZXVOAbSw*i&J)ZePVGiQgU4wc", 9)


# REMOVE ARQUIVOS XLSX DE HOJE
remove(join(default_output, f"casos_{today.strftime('%d_%m_%Y')}.xlsx"))
remove(join(default_output, f"obitos_{today.strftime('%d_%m_%Y')}.xlsx"))
remove(join(default_output, f"casos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"))
remove(join(default_output, f"obitos_regionais_{today.strftime('%d_%m_%Y')}.xlsx"))

<ipython-input-99-c64eec2abfb0>:23: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  pz.compress(path, None, join(default_output, f"{basename(path)}.zip".replace(".xlsx", "")), "Jt\j+ZXVOAbSw*i&J)ZePVGiQgU4wc", 9)
